In [1]:
import os 
import sys
import json 
import pandas as pd 
import re
excel_path = "../data/Data_Mortgage.xlsx"
data_path = "../data/Audio_data.json"

### Data Mortgage 

In [ ]:
excel_data = pd.read_excel(excel_path)
excel_data.dropna(subset='Opportunity',inplace=True)
excel_data.head()

In [ ]:
### Read The Json data of recordings
with open(data_path,'r+') as fd:
    audio_data = json.load(fd)
fd.close()

### Extract all the records for Sales Executive

For example : ABC had converastions with multiple customers so giving this record to data Loader 

In [ ]:
name_of_executive = 'Juraira Manzoor'
df_executive = excel_data[excel_data['Assigned']==name_of_executive]
print(df_executive.shape)
df_executive.head()


In [ ]:
conversation_dict = dict()

conversation_dict['data']=list()
Conversation = []
customers = []

for records in audio_data['Data']:
    if records['content']['Assigned'] == name_of_executive:
        text = re.sub("\s\s+", " ", records['content']['text'])
        customer = records['content']['Customer']
        text = "conversation with Customer {}:{}".format(customer,text)

        conversation_dict['data'].append({
                'name':name_of_executive,
                'text':text,
                'customer':records['content']['Customer'],
                'date':''
            })

with open("../data/tmp.json",'w+') as fc:
    json.dump(conversation_dict,fc,indent=4)

# with open('../data/tmp.txt','w+') as fp:
#     for lines,name in zip(Conversation,customers):
#         lines = re.sub("\s\s+", " ", lines)
#         fp.writelines("Conversation with customer {} :{}\n".format(name,lines))


### Load data as JsonLoader

In [5]:
from langchain.document_loaders import JSONLoader,TextLoader
from langchain.text_splitter import CharacterTextSplitter
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata['customer'] = record.get('customer')
    metadata['date'] = record.get('date')
    return metadata

loader = JSONLoader(
    file_path='../data/tmp.json',
    jq_schema='.data[]',
    content_key="text",
    metadata_func=metadata_func
)
texts = loader.load()


### Load Embeddings to Vector Database

In [1]:
from langchain.embeddings.base import Embeddings
from typing import List
from sentence_transformers import SentenceTransformer
from langchain.embeddings.openai import OpenAIEmbeddings

import openai
import os 
os.environ['OPENAI_API_KEY'] = "" 

In [2]:
class LocalHuggingFaceEmbeddings(Embeddings):
    def __init__(self, model_id): 
        # Should use the GPU by default
        self.model = SentenceTransformer(model_id)
        
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed a list of documents using a locally running
           Hugging Face Sentence Transformer model
        Args:
            texts: The list of texts to embed.
        Returns:
            List of embeddings, one for each text.
        """
        embeddings =self.model.encode(texts)
        return embeddings

    def embed_query(self, text: str) -> List[float]:
        """Embed a query using a locally running HF 
        Sentence trnsformer. 
        Args:
            text: The text to embed.
        Returns:
            Embeddings for the text.
        """
        embedding = self.model.encode(text)
        return list(map(float, embedding))

# local_embeddings = LocalHuggingFaceEmbeddings('multi-qa-mpnet-base-dot-v1')

In [6]:
from langchain.vectorstores import FAISS
vectorstore =  FAISS.from_documents(documents=texts,
                                    embedding = OpenAIEmbeddings()
                                    ) # turn dcos into Vectors and store them in RAM also add metadata 

vectorstore.save_local('../data/faiss_dmac_gpt')

### Chat Query 

In [2]:
import os
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

chat = OpenAI(temperature=0)
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
vectordb = FAISS.load_local('../data/faiss_dmac_gpt/',embeddings=OpenAIEmbeddings())


In [3]:
question_generator = LLMChain(llm=chat, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(chat,chain_type='map_reduce')

chain = ConversationalRetrievalChain(retriever=vectordb.as_retriever(),question_generator=question_generator,
                                           combine_docs_chain=doc_chain
                                           )
chat_history = []


In [4]:
query = "What is Majeed Interested in? "
result = chain({"question": query, "chat_history": chat_history})

print("query:{}".format(result['question']))
print("response:\n{}".format(result['answer']))
chat_history.append((query,result['answer']))

query:What is Majeed Interested in? 
response:
 Majeed is interested in taking a mortgage.


In [5]:
query = "How much mortage is he talking about ?"
result = chain({"question": query, "chat_history": chat_history})
print("query:{}".format(result['question']))
print("response:\n{}".format(result['answer']))
chat_history.append((query,result['answer']))

query:How much mortage is he talking about ?
response:
 Majeed is interested in taking a mortgage between 850,000 and 900,000.


In [7]:
query = "What are the next steps for him ?"
result = chain({"question": query, "chat_history": chat_history})
print("query:{}".format(result['question']))
print("response:\n{}".format(result['answer']))
chat_history.append((query,result['answer']))

query:What are the next steps for him ?
response:
 Majeed's next steps regarding his mortgage are to provide the necessary documents to the mortgage advisor, who will then provide multiple offers from different banks for Majeed to choose from.


In [8]:
query = "What is the value he looking for ?"
result = chain({"question": query, "chat_history": chat_history})
print("query:{}".format(result['question']))
print("response:\n{}".format(result['answer']))
chat_history.append((query,result['answer']))

query:What is the value he looking for ?
response:
 Majeed is interested in a mortgage value between 850,000 and 900,000.


In [8]:
# from glob import glob 

# filenames = glob("../data/Processed_data/*.json")


# def merge_JsonFiles(filename):
#     result = {'conversation data':[]}
#     for f1 in filename:
#         with open(f1, 'r') as infile:
#             data = json.load(infile)
        
#         result['conversation data'].append(data['data'])

#     with open('counseling3.json', 'w') as output_file:
#         json.dump(result, output_file,indent=4)

# merge_JsonFiles(filenames)
# audio_data = "../data/Audio_data_2.json"
# with open(audio_data,'r') as fd :
#     data = json.load(fd)

# conversation_dict = dict()

    
